In [2]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
dfr = hd.read_raw_data("777")

In [46]:
dft = hd.read_txn_data("777")

Time for read_txn_data                 : 2.56 seconds


In [38]:
dfa = hd.read_analysis_data("777")

Time for read_analysis_data            : 0.80 seconds


In [39]:
dfa.isna().sum()

user_id                   0
ym                        0
txns_count                0
txns_volume               0
txn_count_sa              0
txn_count_ca              0
nunique_tag               0
nunique_tag_auto          0
nunique_merchant          0
prop_credit             203
month_income              0
year_income               0
has_regular_income        0
has_month_income          0
age                     560
is_female               224
sa_inflows                0
sa_outflows               0
sa_netflows               0
has_sa_inflows            0
has_reg_sa_inflows        0
has_benefits              0
has_pension               0
has_rent_payment          0
has_mortgage_payment      0
has_loan_funds            0
has_loan_repmt            0
region_name              57
is_urban                166
spend_finance           203
spend_services          203
spend_other_spend       203
spend_household         203
spend_motor             203
spend_communication     203
spend_retail        